<a href="https://colab.research.google.com/github/danie1sung/my-first-repository/blob/main/Simplex_Algorithm5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import math
import numpy as np

def to_tableau(c, A, b):
    idmatnp = np.identity(len(A))
    idmat = idmatnp.tolist()
    xb = [eq + id + [x] for eq, id, x in zip(A, idmat, b)]   #제약조건 좌변 계수 타블루와 우변 타블루 병합
    z = c + [0]*(len(A)+1)                                   #목적함수 좌변 계수 타블루와 0 병합
    return xb + [z]                                          #제약조건 및 목적함수 계수 타블루 병합

def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])       #목적함수값이 개선 가능한지 판단. 최소화 문제에서는 x < 0

def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)             #목적함수 행에서 음수를 가지는 변수를 진입변수로 지정
    restrictions = []
    for eq in tableau[:-1]:                                             #지정한 열에서 최소비율 테스트를 통해 최소비율 지정
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column                                                     #피봇원소 위치 반환

def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]

    i, j = pivot_position
    pivot_value = tableau[i][j]                                       #피봇원소 지정
    new_tableau[i] = np.array(tableau[i]) / pivot_value               #피봇행의 각 항을 피봇원수로 나눔

    for eq_i, eq in enumerate(tableau):                               #피봇연산 실행
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier  #각 항에서 진입변수항을 소거

    return new_tableau                                                #피봇과정을 거친 새로운 타블루 반환


def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1 #기저변수인지 판별

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)          #기저변수의 위치 저장
            solution = columns[-1][one_index]             #기저변수값 저장
        solutions.append(solution)

    return solutions                                      #최종 기저가능해 반환

def simplex(c, A, b):
    tableau = to_tableau(c, A, b)
    count = 0
    while can_be_improved(tableau):                       #목적함수값 개선 가능 여부 판단
        count += 1
        pivot_position = get_pivot_position(tableau)      #진입변수, 피봇원소 지정
        tableau = pivot_step(tableau, pivot_position)     #피봇연산 수행

    return get_solution(tableau), count                          #최종 기저가능해 반환



c = [ 2, 1, -6, 0, 1, 2, -5, 0, 2, 4, 5, 0, 9, 9, 10]
A = [
    [ 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #19
    [ 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [ 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
    [ 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],

]

b = [10, 30, 10, 15, 10, 15]
print(simplex(c,A,b))



([0, 0, 0, 10.0, 0, 0, 0, 30.0, 0, 0, 0, 10.0, 15.0, 10.0, 15.0, 10.0, 30.0, 10.0, 0, 0, 0], 13)
